In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import *
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from haversine import haversine
from datetime import *

In [2]:
busan_anch_train_final = pd.read_csv("../기상청공모전/data/busan_anch_train_final.csv",encoding = "utf-8", index_col=0)
busan = busan_anch_train_final.copy()

col_name ={'busan_anch_train_final.num':'num',
      'busan_anch_train_final.time':'time',
      'busan_anch_train_final.latitude':'latitude',
      'busan_anch_train_final.longitude':'longitude',
      'busan_anch_train_final.sog':'sog',
      'busan_anch_train_final.cog':'cog',
      'busan_anch_train_final.hdg':'hdg'}

busan.rename(columns=col_name, inplace=True)

busan.time = busan.time.str[1:-1]
busan.time = pd.to_datetime(busan.time)

busan.latitude = busan.latitude.str[2:-1]
busan.longitude = busan.longitude.str[2:-1]

busan.hdg = busan.hdg.str[1:-1]

lt = list(busan[busan.hdg=='-99.-'].index)
for i in lt:
    busan.loc[i,'hdg']='-99.9'
    
busan = busan.astype({'latitude':'float','longitude':'float', 'hdg':'float'})

busan['anchor_drag'] = 0

busan.to_csv('../기상청공모전/data/busan_anch.csv', encoding = "utf-8", index=False)

In [3]:
ulsan_anch_train_final = pd.read_csv("../기상청공모전/data/ulsan_anch_train_final.csv",encoding = "utf-8", index_col=0)
ulsan = ulsan_anch_train_final.copy()

# 컬럼명 변경
col_name ={'ulsan_anch_train_final.num':'num',
      'ulsan_anch_train_final.time':'time',
      'ulsan_anch_train_final.latitude':'latitude',
      'ulsan_anch_train_final.longitude':'longitude',
      'ulsan_anch_train_final.sog':'sog',
      'ulsan_anch_train_final.cog':'cog',
      'ulsan_anch_train_final.hdg':'hdg'}

ulsan.rename(columns=col_name, inplace=True)

# 위도 경도 데이터 실수화
ulsan.time = ulsan.time.str[1:-1]
ulsan.time = pd.to_datetime(ulsan.time)
ulsan.latitude = ulsan.latitude.str[2:-1]
ulsan.longitude = ulsan.longitude.str[2:-1]

ulsan = ulsan.astype({'latitude':'float','longitude':'float'})

# 답안 컬럼 추가
ulsan['anchor_drag'] = 0

# 데이터 저장
ulsan.to_csv('../기상청공모전/data/ulsan_anch.csv', encoding = "utf-8", index=False)

In [4]:
busan_drag_train_final = pd.read_csv("../기상청공모전/data/busan_drag_train_final.csv",encoding = "utf-8", index_col=0)
busan_answer = pd.read_csv("../기상청공모전/data/busan_answer.csv",encoding = "utf-8", index_col=0)

busan = busan_drag_train_final.copy()
banswer = busan_answer.copy()

col_name ={'busan_drag_train_final.num':'num',
      'busan_drag_train_final.time':'time',
      'busan_drag_train_final.latitude':'latitude',
      'busan_drag_train_final.longitude':'longitude',
      'busan_drag_train_final.sog':'sog',
      'busan_drag_train_final.cog':'cog',
      'busan_drag_train_final.hdg':'hdg'}

busan.rename(columns=col_name, inplace=True)

busan.latitude = busan.latitude.str[2:-1]
busan.longitude = busan.longitude.str[2:-1]

busan = busan.astype({'latitude':'float','longitude':'float'})

banswer['busan_answer.lat'] = banswer['busan_answer.lat'].str[2:-1]
banswer['busan_answer.lon'] = banswer['busan_answer.lon'].str[2:-1]

banswer = banswer.astype({'busan_answer.lat':'float','busan_answer.lon':'float'})

busan.time = busan.time.str[1:-1]
busan.time = pd.to_datetime(busan.time)

i_col2 = list(busan['num'].unique())
anchor_drag2 = [0 for i in range(len(i_col2))]

busan['anchor_drag'] = 0

nowhere_busan = []
for i in i_col2:
    col = banswer[banswer['busan_answer.num']==i]
    n_df = busan[(busan['latitude']==col['busan_answer.lat'].values[0])&(busan['longitude']==col['busan_answer.lon'].values[0])]
    if len(n_df.index)==0:
        nowhere_busan.append(i)

for i in i_col2:
    col = banswer[banswer['busan_answer.num']==i]
    
    if i in nowhere_busan:
        n_df = busan[(busan.num==i)&
                     (busan.time.dt.year == col['busan_answer.year'].values[0])&
                     (busan.time.dt.month == col['busan_answer.mon'].values[0])&
                     (busan.time.dt.day == col['busan_answer.day'].values[0])&
                     (busan.time.dt.hour == col['busan_answer.hour'].values[0])&
                     (busan.time.dt.minute == col['busan_answer.min'].values[0])]
        start = n_df.index[0]
        
    else:
        n_df = busan[(busan['latitude']==col['busan_answer.lat'].values[0])&
                              (busan['longitude']==col['busan_answer.lon'].values[0])]
        start = n_df.index[0]
    
    end = busan[busan['num']==i].index[-1]
    
    busan.loc[start:end, 'anchor_drag']=1

busan.to_csv('../기상청공모전/data/busan_drag&answer.csv', encoding = "utf-8", index=False)

In [5]:
ulsan_drag_train_final = pd.read_csv("../기상청공모전/data/ulsan_drag_train_final.csv",encoding = "utf-8", index_col=0)
ulsan_answer = pd.read_csv("../기상청공모전/data/ulsan_answer.csv",encoding = "utf-8", index_col=0)
ulsan = ulsan_drag_train_final.copy()

col_name ={'ulsan_drag_train_final.num':'num',
      'ulsan_drag_train_final.time':'time',
      'ulsan_drag_train_final.latitude':'latitude',
      'ulsan_drag_train_final.longitude':'longitude',
      'ulsan_drag_train_final.sog':'sog',
      'ulsan_drag_train_final.cog':'cog',
      'ulsan_drag_train_final.hdg':'hdg'}

ulsan.rename(columns=col_name, inplace=True)

ulsan.latitude = ulsan.latitude.str[2:-1]
ulsan.longitude = ulsan.longitude.str[2:-1]

ulsan = ulsan.astype({'latitude':'float','longitude':'float'})


ulsan.time = ulsan.time.str[1:-1]
ulsan.time = pd.to_datetime(ulsan.time)

ulsan['anchor_drag'] = 0

uanswer = ulsan_answer.copy()
uanswer['ulsan_answer.lat'] = uanswer['ulsan_answer.lat'].str[2:-1]
uanswer['ulsan_answer.lon'] = uanswer['ulsan_answer.lon'].str[2:-1]
uanswer = uanswer.astype({'ulsan_answer.lat':'float','ulsan_answer.lon':'float'})

i_col = list(ulsan_drag_train_final['ulsan_drag_train_final.num'].unique())
nowhere_ulsan = []
for i in i_col:
    col = uanswer[uanswer['ulsan_answer.num']==i]
    n_df = ulsan[(ulsan['latitude']==col['ulsan_answer.lat'].values[0])&(ulsan['longitude']==col['ulsan_answer.lon'].values[0])]
    if len(n_df.index)==0:
        nowhere_ulsan.append(i)

for i in i_col:
    col = uanswer[uanswer['ulsan_answer.num']==i]
    
    if i in nowhere_ulsan:                       
        n_df = ulsan[(ulsan.num==i)&
                     (ulsan.time.dt.year == col['ulsan_answer.year'].values[0])&
                     (ulsan.time.dt.month == col['ulsan_answer.mon'].values[0])&
                     (ulsan.time.dt.day == col['ulsan_answer.day'].values[0])&
                     (ulsan.time.dt.hour == col['ulsan_answer.hour'].values[0])&
                     (ulsan.time.dt.minute == col['ulsan_answer.min'].values[0])]
        start = n_df.index[0]
        
    else:
        n_df = ulsan[(ulsan['latitude']==col['ulsan_answer.lat'].values[0])&
                                      (ulsan['longitude']==col['ulsan_answer.lon'].values[0])]
        start = n_df.index[0]
        
    end = ulsan[ulsan['num']==i].index[-1]
    
    ulsan.loc[start:end, 'anchor_drag']=1

ulsan.to_csv('../기상청공모전/data/ulsan_drag&answer.csv', encoding = "utf-8", index=False)

In [6]:
ulsan_anch = pd.read_csv("../기상청공모전/data/ulsan_anch.csv",encoding = "utf-8")
busan_anch = pd.read_csv("../기상청공모전/data/busan_anch.csv",encoding = "utf-8")
ulsan_drag = pd.read_csv("../기상청공모전/data/ulsan_drag&answer.csv",encoding = "utf-8")
busan_drag = pd.read_csv("../기상청공모전/data/busan_drag&answer.csv",encoding = "utf-8")

ulsan_anch.num = ulsan_anch.num+1000
busan_anch.num = busan_anch.num+2000
ulsan_drag.num = ulsan_drag.num+3000
busan_drag.num = busan_drag.num+4000

total_data= pd.concat([ulsan_anch,busan_anch,ulsan_drag,busan_drag]).reset_index(drop=True)
total_data.to_csv('../기상청공모전/data/total_data.csv', index=False)

In [7]:
_order = ["anchor_drag", "num", "time", "latitude", "longitude","sog","cog","hdg"]
total = total_data.reindex(columns = _order)

In [8]:
for i in range(1,len(total)):
    total.loc[i,'d_lati'] = total.loc[i,'latitude'] - total.loc[i-1,'latitude']
    total.loc[i,'d_long'] = total.loc[i,'longitude'] - total.loc[i-1,'longitude']
    
for i in range(1,len(total)):
    total.loc[i,'d_d_lati'] = total.loc[i,'d_lati'] - total.loc[i-1,'d_lati']
    total.loc[i,'d_d_long'] = total.loc[i,'d_lati'] - total.loc[i-1,'d_lati']

In [9]:
s1 = -99.9
s2 = -99

total["cog"] = total["cog"].replace(s1,float('nan'))
total["hdg"] = total["hdg"].replace(s1,float('nan'))
total["hdg"] = total["hdg"].replace(s2,float('nan'))

total = total.interpolate(method = "quadratic")

In [10]:
total['time_gap'] = 0

for i in range(1, len(total)):
    if total.loc[i, 'time_gap'] == 0:
        dt = (pd.to_datetime(total.loc[i, 'time']) - pd.to_datetime(total.loc[i-1, 'time'])).total_seconds()
        if dt != 0:
            total.loc[i, 'time_gap'] = dt
        else:
            a = total.loc[(total['num'] == total.loc[i, 'num']) & (total['time'] == total.loc[i, 'time'])].index.tolist()
            total.loc[a, 'time_gap'] = 1 / len(a)

In [11]:
total.to_csv("preprocessing1.csv", index=False)

In [12]:
total = pd.read_csv("../기상청공모전/data/preprocessing1.csv")

In [13]:
for i in range(1, len(total)):
    total.loc[i, 'km/s'] = haversine((total.loc[i, 'latitude'],total.loc[i, 'longitude']),(total.loc[i-1, 'latitude'],total.loc[i-1, 'longitude']),unit = 'km')

for i in range(2, len(total)):
    total.loc[i, 'km/s_2'] = (total.loc[i, 'km/s'] - total.loc[i-1, 'km/s']) / total.loc[i,'time_gap']

In [15]:
# 각 배 번호의 1번째 2번째 삭제
ship_num = total.iloc[:, 1].unique()

_drop = []
for i in ship_num:
    k = total[total["num"] == i].index[0]
    _drop.append(k)
    k += 1
    _drop.append(k)

total = total[~total.index.isin(_drop)]

total = total[['anchor_drag', 'num', 'time', 'latitude', 'longitude', 'sog', 'cog', 'hdg', 'time_gap', 'km/s', 'km/s_2', 'd_lati', 'd_long', 'd_d_lati', 'd_d_long']]
total.columns = ['anchor_drag','num', 'time', 'latitude', 'longitude', 'sog', 'cog', 'hdg', 'time_gap', 'd_km/s', 'dd_km/s', 'd_lati', 'd_long', 'd_d_lati', 'd_d_long']
total = total.reset_index(drop=True)

In [17]:
ship_num = total[total['num']>3000]['num'].unique()

drop_list = []

for i in ship_num:
    drop_index = []
    drop_index = total[(total['num']==i) & (total["anchor_drag"] == 1)].index.tolist()
    if not drop_index:
        continue
    else:
        drop_list.extend(drop_index)
        drop_list.remove(min(drop_index))
total = total[~total.index.isin(drop_list)]

total = total.reset_index(drop=True)

In [18]:
total.to_csv("preprocessing2.csv",index=False)

In [19]:
total = pd.read_csv("../기상청공모전/data/preprocessing2.csv")

In [20]:
total

,anchor_drag,num,time,latitude,longitude,sog,cog,hdg,time_gap,d_km/s,dd_km/s,d_lati,d_long,d_d_lati,d_d_long
0,0,1001,2021-01-03 11:14:38,35.461472,129.439878,0.4,9.6,273.0,182.0,0.045969,-0.000152,0.000312,0.000333,0.000342,0.000342
1,0,1001,2021-01-03 11:17:33,35.461688,129.439792,0.2,328.0,267.0,175.0,0.025250,-0.000118,0.000216,-0.000086,-0.000096,-0.000096
2,0,1001,2021-01-03 11:20:37,35.461708,129.439733,0.2,260.1,265.0,184.0,0.005788,-0.000106,0.000020,-0.000059,-0.000196,-0.000196
3,0,1001,2021-01-03 11:23:37,35.461720,129.439708,0.1,262.8,270.0,180.0,0.002628,-0.000018,0.000012,-0.000025,-0.000008,-0.000008
4,0,1001,2021-01-03 11:26:38,35.461733,129.439717,0.3,277.6,273.0,181.0,0.001660,-0.000005,0.000013,0.000009,0.000001,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667112,0,4087,2022-05-07 20:20:00,35.052933,129.060633,0.9,141.0,99.0,0.2,0.006319,0.007453,-0.000050,0.000033,-0.000016,-0.000016
667113,0,4087,2022-05-07 20:20:00,35.052900,129.060667,1.0,145.0,98.0,0.2,0.004800,-0.007595,-0.000033,0.000034,0.000017,0.000017
667114,0,4087,2022-05-07 20:20:00,35.052850,129.060700,1.1,148.0,97.0,0.2,0.006319,0.007595,-0.000050,0.000033,-0.000017,-0.000017
667115,0,4087,2022-05-07 20:20:00,35.052750,129.060767,1.2,152.0,94.0,0.2,0.012682,0.031814,-0.000100,0.000067,-0.000050,-0.000050


In [21]:
bs_test = pd.read_csv("../기상청공모전/data/test_busan_ggg.csv")
bs_test = bs_test.iloc[:,1:]

us_test = pd.read_csv("../기상청공모전/data/test_ulsan_ggg.csv")
us_test = us_test.iloc[:,1:]

In [22]:
df = total.copy()
del df['time']

df_1 = df[df['num']<3000]
df_2 = df[df['num']>=3000]

In [23]:
X_1_train = df_1.iloc[:,1:]
y_1_train = df_1.iloc[:,0]

X_2_train = df_2.iloc[:,1:]
y_2_train = df_2.iloc[:,0]

In [24]:
X_2_samp, y_2_samp = SMOTE(random_state=4).fit_resample(X_2_train, y_2_train)

In [25]:
X_2_samp['anchor_drag'] = y_2_samp

In [26]:
total_samp = X_2_samp.copy()
total_samp = total_samp.sort_values('num')

total_nonsamp = X_1_train.copy()
total_nonsamp['anchor_drag'] = y_1_train

In [27]:
total_new = pd.concat([total_nonsamp, total_samp])
total

,anchor_drag,num,time,latitude,longitude,sog,cog,hdg,time_gap,d_km/s,dd_km/s,d_lati,d_long,d_d_lati,d_d_long
0,0,1001,2021-01-03 11:14:38,35.461472,129.439878,0.4,9.6,273.0,182.0,0.045969,-0.000152,0.000312,0.000333,0.000342,0.000342
1,0,1001,2021-01-03 11:17:33,35.461688,129.439792,0.2,328.0,267.0,175.0,0.025250,-0.000118,0.000216,-0.000086,-0.000096,-0.000096
2,0,1001,2021-01-03 11:20:37,35.461708,129.439733,0.2,260.1,265.0,184.0,0.005788,-0.000106,0.000020,-0.000059,-0.000196,-0.000196
3,0,1001,2021-01-03 11:23:37,35.461720,129.439708,0.1,262.8,270.0,180.0,0.002628,-0.000018,0.000012,-0.000025,-0.000008,-0.000008
4,0,1001,2021-01-03 11:26:38,35.461733,129.439717,0.3,277.6,273.0,181.0,0.001660,-0.000005,0.000013,0.000009,0.000001,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667112,0,4087,2022-05-07 20:20:00,35.052933,129.060633,0.9,141.0,99.0,0.2,0.006319,0.007453,-0.000050,0.000033,-0.000016,-0.000016
667113,0,4087,2022-05-07 20:20:00,35.052900,129.060667,1.0,145.0,98.0,0.2,0.004800,-0.007595,-0.000033,0.000034,0.000017,0.000017
667114,0,4087,2022-05-07 20:20:00,35.052850,129.060700,1.1,148.0,97.0,0.2,0.006319,0.007595,-0.000050,0.000033,-0.000017,-0.000017
667115,0,4087,2022-05-07 20:20:00,35.052750,129.060767,1.2,152.0,94.0,0.2,0.012682,0.031814,-0.000100,0.000067,-0.000050,-0.000050


In [28]:
X_train = total_new.iloc[:,1:-1]
y_train = total_new.iloc[:,-1]
X_test_bs = bs_test.iloc[:,2:]
X_test_us = us_test.iloc[:,2:]

In [30]:
# XGBoost 데이터셋 생성
dtrain = xgb.DMatrix(X_train_samp, label=y_train_samp)
dtest_bs = xgb.DMatrix(X_test_bs)
dtest_us = xgb.DMatrix(X_test_us)

# XGBoost 모델 학습
params = {'objective': 'binary:logistic', 'eval_metric': 'logloss'}
num_rounds = 100  # 반복 횟수 설정

model = xgb.train(params, dtrain, num_rounds)

# 테스트 데이터 예측
y_pred_bs = model.predict(dtest_bs)
y_pred_us = model.predict(dtest_us)
y_pred_binary_bs = [1 if y >= 0.5 else 0 for y in y_pred_bs]
y_pred_binary_us = [1 if y >= 0.5 else 0 for y in y_pred_us]

In [31]:
result = pd.read_excel("../기상청공모전/data/oceanUser.xlsx")

In [32]:
temp1 = bs_test.copy()
temp2 = us_test.copy()
temp1['time'] = pd.to_datetime(temp1['time'])
temp2['time'] = pd.to_datetime(temp2['time'])

temp1['anchor_drag'] = y_pred_binary_bs
temp2['anchor_drag'] = y_pred_binary_us

In [33]:
busan_target=[]
ulsan_target=[]
for i in range(len(temp1)):
    if temp1.iloc[i,-1] == 1:
        busan_target.append(i)
for i in range(len(temp2)):
    if temp2.iloc[i,-1] == 1:
        ulsan_target.append(i)

In [34]:
temp1["HOUR"] = temp1["time"].dt.hour
temp1["MIN"] = temp1["time"].dt.minute

temp2["HOUR"] = temp2["time"].dt.hour
temp2["MIN"] = temp2["time"].dt.minute

In [35]:
bs_final = []
us_final = []
for i in busan_target:
    bs_final.append([temp1.iloc[i,0], temp1.iloc[i,-2], temp1.iloc[i,-1]])    
for i in ulsan_target:
    us_final.append([temp2.iloc[i,0], temp2.iloc[i,-2], temp2.iloc[i,-1]])

for i in range(len(result)):
    if result.iloc[i,0] == "BUSAN":
        if [result.iloc[i,1], result.iloc[i,5], result.iloc[i,6]] in bs_final:
            result.iloc[i,-1] = 1        
    else:
        if [result.iloc[i,1], result.iloc[i,5], result.iloc[i,6]] in us_final:
            result.iloc[i,-1] = 1 

In [36]:
result["TARGET"] = result["TARGET"].fillna(0)
result["TARGET"] = result["TARGET"].astype(int)

In [37]:
bs = result.loc[result["AREA"] == "BUSAN"]
us = result.loc[result["AREA"] == "ULSAN"]

In [38]:
bs_index = []
us_index = []
for i in range(len(bs)):
    if bs.iloc[i,-1] == 1:
        bs_index.append(i)
for i in range(len(us)):
    if us.iloc[i,-1] == 1:
        us_index.append(i)

In [39]:
bs_ship = []
us_ship = []
for i in bs_index:
    k = bs.iloc[i,1]
    if k not in bs_ship:
        bs_ship.append(k)
for i in us_index:
    k = us.iloc[i,1]
    if k not in us_ship:
        us_ship.append(k)

In [40]:
for i in range(len(bs)):
    for j in bs_ship:
        if bs.iloc[i,1] == j:
            if bs.iloc[i,-1] == 1:
                bs.iloc[i+1:i+60,-1] = 0
                
c = 0
for i in range(len(bs)):
    if bs.iloc[i,-1] == 1:
        c +=1

In [41]:
for i in range(len(us)):
    for j in us_ship:
        if us.iloc[i,1] == j:
            if us.iloc[i,-1] == 1:
                us.iloc[i+1:i+60,-1] = 0
c = 0
for i in range(len(us)):
    if us.iloc[i,-1] == 1:
        c +=1

In [42]:
bs_final_index = []
us_final_index = []
for i in range(len(bs)):
    if bs.iloc[i,-1] == 1:
        bs_final_index.append(i)
for i in range(len(us)):
    if us.iloc[i,-1] == 1:
        us_final_index.append(i)

In [43]:
for i in range(len(us_final_index)):
    us_final_index[i] += len(bs)

In [44]:
result["TARGET"] = 0

In [45]:
for i in bs_final_index:
    result.iloc[i,-1] = 1
for i in us_final_index:
    result.iloc[i,-1] = 1

c = 0
for i in range(len(result)):
    if result.iloc[i,-1] == 1:
        c += 1

In [46]:
result.to_excel("230100 (3-2 + reduced 1).xlsx",sheet_name = "OCEAN", index=False)